# Setup Environment, Load Data

In [ ]:
import smtplib
from email.mime.text import MIMEText

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail, change if using Outlook or another service
SMTP_PORT = 465  # SSL port (or use 587 for TLS)
SENDER_EMAIL = "abhaymathur1000@gmail.com"  # Replace with your email
SENDER_PASSWORD = "blom axpb quot zcdv"  # Use the generated app password
RECEIVER_EMAIL = "abhaymathur1000@gmail.com"  # Your email (or another recipient)

def send_email(subject, body=""):
    """Sends an email notification"""
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")



# Notify when the script completes
# send_email("✅ Script Completed", "Your Python script has finished running.")


In [ ]:
#Import Libraries
import os
import json
import csv
import matplotlib.pyplot as plt
import sklearn
import tensorflow
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import torch
import torch.nn as nn
import time
import spacy
import kagglehub

#NLP Packages
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from nltk.corpus import stopwords
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from IPython.display import display, clear_output

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
if os.path.exists("/kaggle"):
    path = "/kaggle/input/yelp-dataset"
else:
    path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")
print("Path to dataset files:", path)



def load_data(path, filename, chunk_size=10000):
    chunks = pd.read_json(os.path.join(path, filename), lines=True, chunksize=chunk_size)

    df_list = []

    for chunk in chunks:
        df_list.append(chunk)


    return pd.concat(df_list, ignore_index=True)



print("\nDataset Load Times:\n")
start_time = time.time()
businesses_df = load_data(path, "yelp_academic_dataset_business.json")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Business Load Time: {elapsed_time:.4f} seconds")

start_time = time.time()
tips_df = load_data(path, "yelp_academic_dataset_tip.json")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tips Load Time: {elapsed_time:.4f} seconds")

start_time = time.time()
reviews_df = load_data(path, "yelp_academic_dataset_review.json")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Review Load Time: {elapsed_time:.4f} seconds")
num_rev_load = len(reviews_df)



# Clean Data

Current Thoughts:
* Should we be counting categories weighted based on reviews, or should we just count categories by businesses?
* Do we need to lemmatize for a BERT model considering it has been trained? Also, can lemmatized words be tokenized?

In [ ]:
#Filter out Data that Belongs to Businesses Under a Certain Threshold
minimum_business_reviews = 30
reviews_df = reviews_df[reviews_df['business_id'].map(reviews_df['business_id'].value_counts()).gt(minimum_business_reviews)]

#Create Merged DataFrame of Remaining Reviewed Businesses, and Split Categories into List
df_rb = pd.merge(reviews_df, businesses_df, on="business_id")
df_rb['categories'] = df_rb['categories'].str.split(", ")

#Print Metrics
print(f"\nNumber of Total Businesses Represented: {len(df_rb['business_id'].unique())}")
print(f"Number of Total Reviews: {len(df_rb)}")
print(f"Percentage of Reviews Kept: {len(df_rb)/num_rev_load*100}%")

In [ ]:
#Dictionary to Store Categories, and Variables to Report Later
all_unique_cats = {}
total_cats_before = df_rb['categories'].apply(len).sum()
len_before = len(df_rb)

#Iterate through each Business' Categories and Count Occurances (Weighted Based on Reviews)
for cats in df_rb['categories']:
    for cat in cats:
        if cat in all_unique_cats:
            all_unique_cats[cat] += 1
        else:
            all_unique_cats[cat] = 1

#Number of Top Categories to Keep
num_cats = 30
sorted_cats = sorted(all_unique_cats.items(), key=lambda x: x[1], reverse=True)

#Select Top Categories
top_cats = [x[0] for x in sorted_cats[:num_cats]]

#Remove Reviews that Belong to Businesses with None of the Top Categories
df_rb = df_rb[df_rb['categories'].apply(lambda x: any(cat in top_cats for cat in x) if x else False)]

#Remove Categories from Remaining Reviews that are not in the Top Categories
df_rb['categories'] = df_rb['categories'].apply(lambda x: ([i for i in x if i in top_cats]))

#Count Total Number of Categories
total_cats_after = df_rb['categories'].apply(len).sum()

print(f"\nTotal Initial Unique Categories: {len(all_unique_cats)}")
print(f"Number of Total Reviews: {len(df_rb)}")
print(f"Percentage of Reviews Kept: {len(df_rb)/num_rev_load*100}%")
print(f"Average Number of Categories Before: {total_cats_before/len_before}")
print(f"Average Number of Categories After: {total_cats_after/len(df_rb)}")

In [ ]:
#Clean review text field
import nltk
nltk.download('stopwords')
sw = stopwords.words('english')
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z?.,!¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    html = re.compile(r'<.*?>')
    text = html.sub(r'',text) #Removing html tags
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^,' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text) #removing stopwords
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    return text

df_rb['text'] = df_rb['text'].apply(lambda x: clean_text(x))

In [ ]:
# Load the spaCy English model

nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])


def lemmatize(text):
    # Process the text using spaCy
    start_time = time.time()
    doc = nlp(text)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Step 1 Load Time: {elapsed_time:.4f} seconds")
    
    # Extract lemmatized tokens
    start_time = time.time()
    lemmatized_tokens = [token.lemma_ for token in doc]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Step 2 Load Time: {elapsed_time:.4f} seconds")
    
    # Join the lemmatized tokens into a sentence
    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text

#Uncomment when running on better computer
df_rb['text'] = df_rb['text'].apply(lambda x: lemmatize(x))

In [ ]:
save_path = os.path.join('data', 'reviews.parquet')

df_rb.to_parquet(save_path)
print("Parquet file size: ", os.path.getsize(save_path)/(1024*1024), "MB")

start_time = time.time()
df_rb_read = pd.read_parquet(save_path)
read_time = time.time() - start_time
print("It took ", read_time, "seconds to read the parquet file")
print("DF shape: ", df_rb_read.shape)